In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path_to_description = '/content/drive/MyDrive/Colab Notebooks/wine_description.txt'
path_to_train = '/content/drive/MyDrive/Colab Notebooks/wine_training.csv'
path_to_test = '/content/drive/MyDrive/Colab Notebooks/wine_test.csv'
path_to_new_batch = '/content/drive/MyDrive/Colab Notebooks/wine_new_batch.csv'

d_train = pd.read_csv(path_to_train, names=["fixed acidity", "volatile acidity", "citric acid", 'residual sugar', "chlorides", "free sulfur dioxide",
   "total sulfur dioxide", "density", "pH", "sulphates", "alcohol", "quality"])
d_test = pd.read_csv(path_to_test, names=["fixed acidity", "volatile acidity", "citric acid", 'residual sugar', "chlorides", "free sulfur dioxide",
   "total sulfur dioxide", "density", "pH", "sulphates", "alcohol", "quality"])
d_new = pd.read_csv(path_to_new_batch, names=["fixed acidity", "volatile acidity", "citric acid", 'residual sugar', "chlorides", "free sulfur dioxide",
   "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"])

In [4]:
train = np.array(d_train)
test = np.array(d_test)
X_new = np.array(d_new)
X_train = train[:,0:11]
Y_train = train[:,11]
print(X_train.shape)
X_test = test[:,0:11]
Y_test = test[:,11]
print(X_test.shape)

(1115, 11)
(479, 11)


In [5]:
import sklearn as skl
import sklearn.kernel_ridge
import numpy.matlib

X_train_N = X_train.shape[0]
X_train_mean = np.mean(X_train, axis=0)
X_train_std = np.std(X_train, axis=0)
X_train_normal = (X_train - np.matlib.repmat(X_train_mean, X_train_N, 1))/np.matlib.repmat(X_train_std, X_train_N, 1)

Y_train_N = Y_train.shape[0]
Y_train_mean = np.mean(Y_train, axis=0)
Y_train_std = np.std(Y_train, axis=0)
Y_train_normal = (Y_train - Y_train_mean)/Y_train_std

X_test_N = X_test.shape[0]
X_test_normal = (X_test - np.matlib.repmat(X_train_mean, X_test_N, 1))/np.matlib.repmat(X_train_std, X_test_N, 1)
Y_test_normal = (Y_test - Y_train_mean)/Y_train_std

X_new_N = X_new.shape[0]
X_new_normal = (X_new - np.matlib.repmat(X_train_mean, X_new_N, 1))/np.matlib.repmat(X_train_std, X_new_N, 1)

In [6]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression().fit(X_train_normal, Y_train_normal)
LR_train_normal = LR.predict(X_train_normal)
LR_test_normal = LR.predict(X_test_normal)

In [29]:
import sklearn.model_selection

sgm = np.linspace(1,3,10)
lmbd = np.linspace(-3,-1,10)
scores = np.zeros((10,10))
RBF_CV = skl.kernel_ridge.KernelRidge(kernel='rbf')

for i in range(10):
  RBF_CV.gamma = 1/(2*(2**sgm[i])**2)
  for j in range(10):
    RBF_CV.alpha = (2**lmbd[j])
    this_score = skl.model_selection.cross_val_score(RBF_CV,X_train_normal,Y_train_normal,scoring='neg_mean_squared_error',cv=10,n_jobs=-1)
    scores[i,j] = (np.mean(this_score))
ij_max = np.array(np.where(scores==scores.max()),dtype=int).flatten()
log_2_sg_rbf = sgm[ij_max[0]]
log_2_lmbd_rbf = lmbd[ij_max[1]]
print('log_2_sg_rbf:', sgm[ij_max[0]], 'log_2_lmbd_rbf: ', lmbd[ij_max[1]])


log_2_sg_rbf: 1.8888888888888888 log_2_lmbd_rbf:  -2.3333333333333335


In [10]:
 # test
import sklearn.model_selection

sgm = np.linspace(1,3,10)
lmbd = np.linspace(-3,-1,10)
scores = np.zeros((10,10))
RBF_CV = skl.kernel_ridge.KernelRidge(kernel='rbf')

for i in range(10):
  RBF_CV.gamma = 1/(2*(2**sgm[i])**2)
  for j in range(10):
    RBF_CV.alpha = (2**lmbd[j])
    this_score = skl.model_selection.cross_val_score(RBF_CV,X_train_normal,Y_train_normal,scoring='neg_mean_squared_error',cv=10,n_jobs=-1)
    scores[i,j] = (np.mean(this_score))
ij_max = np.array(np.where(scores==scores.max()),dtype=int).flatten()
log_2_sg_rbf = sgm[ij_max[0]]
log_2_lmbd_rbf = lmbd[ij_max[1]]
print('log_2_sg_rbf:', sgm[ij_max[0]], 'log_2_lmbd_rbf: ', lmbd[ij_max[1]])

log_2_sg_rbf: 1.8888888888888888 log_2_lmbd_rbf:  -2.3333333333333335


In [11]:
sigma_rbf = 2**log_2_sg_rbf
lambda_rbf = 2**log_2_lmbd_rbf
print(sigma_rbf, lambda_rbf)

3.7034988491491614 0.19842513149602492


In [15]:
alpha_rbf = 2**log_2_lmbd_rbf
gamma_rbf = 1/(2*sigma_rbf**2)
print(alpha_rbf,gamma_rbf)

0.19842513149602492 0.03645403248675365


In [43]:
sgm = np.linspace(1,3,10)
lmbd = np.linspace(-3,-1,10)
scores = np.zeros((10,10))
LAP_CV = skl.kernel_ridge.KernelRidge(kernel='laplacian')

for i in range(10):
  LAP_CV.gamma = 1/(2**sgm[i])
  for j in range(10):
    LAP_CV.alpha = (2**lmbd[j])
    this_score = skl.model_selection.cross_val_score(LAP_CV,X_train_normal,Y_train_normal,scoring='neg_mean_squared_error',cv=10,n_jobs=-1)
    scores[i,j] = (np.mean(this_score))
ij_max = np.array(np.where(scores==scores.max()),dtype=int).flatten()
log_2_sg_lap = sgm[ij_max[0]]
log_2_lmbd_lap = lmbd[ij_max[1]]
print('log_2_sg_lap:', sgm[ij_max[0]], 'log_2_lmbd_lap: ', lmbd[ij_max[1]])

log_2_sg_lap: 2.111111111111111 log_2_lmbd_lap:  -2.111111111111111


In [13]:
# test
sgm = np.linspace(1,3,10)
lmbd = np.linspace(-3,-1,10)
scores = np.zeros((10,10))
LAP_CV = skl.kernel_ridge.KernelRidge(kernel='laplacian')

for i in range(10):
  LAP_CV.gamma = 1/(2**sgm[i])
  for j in range(10):
    LAP_CV.alpha = (2**lmbd[j])
    this_score = skl.model_selection.cross_val_score(LAP_CV,X_train_normal,Y_train_normal,scoring='neg_mean_squared_error',cv=10,n_jobs=-1)
    scores[i,j] = (np.mean(this_score))
ij_max = np.array(np.where(scores==scores.max()),dtype=int).flatten()
log_2_sg_lap = sgm[ij_max[0]]
log_2_lmbd_lap = lmbd[ij_max[1]]
print('log_2_sg_lap:', sgm[ij_max[0]], 'log_2_lmbd_lap: ', lmbd[ij_max[1]])

log_2_sg_lap: 2.111111111111111 log_2_lmbd_lap:  -2.111111111111111


In [14]:
sigma_lap = 2**log_2_sg_lap
lambda_lap = 2**log_2_lmbd_lap
print(sigma_lap, lambda_lap)

4.320238955569224 0.2314686780718226


In [16]:
alpha_lap = 2**log_2_lmbd_lap
gamma_lap = 1/(sigma_lap)
print(alpha_lap,gamma_lap)

0.2314686780718226 0.23146867807182261


In [17]:
from sklearn.metrics import mean_squared_error

LR_train_mse = mean_squared_error(Y_train_normal, LR_train_normal)
LR_test_mse = mean_squared_error(Y_test_normal, LR_test_normal)
print(LR_train_mse, LR_test_mse)

0.6278484956554882 0.7471696905187208


In [18]:
RBF = skl.kernel_ridge.KernelRidge(kernel='rbf',alpha=alpha_rbf,gamma=gamma_rbf)
RBF.fit(X_train_normal, Y_train_normal)
RBF_train_normal = RBF.predict(X_train_normal)
RBF_test_normal = RBF.predict(X_test_normal)

RBF_train_mse = mean_squared_error(Y_train_normal, RBF_train_normal)
RBF_test_mse = mean_squared_error(Y_test_normal, RBF_test_normal)
print(RBF_train_mse, RBF_test_mse)

0.4548788888959536 0.6786661476640042


In [20]:
LAP = skl.kernel_ridge.KernelRidge(kernel='laplacian',alpha=alpha_lap,gamma=gamma_lap)
LAP.fit(X_train_normal, Y_train_normal)
LAP_train_normal = LAP.predict(X_train_normal)
LAP_test_normal = LAP.predict(X_test_normal)

LAP_train_mse = mean_squared_error(Y_train_normal, LAP_train_normal)
LAP_test_mse = mean_squared_error(Y_test_normal, LAP_test_normal)
print(LAP_train_mse, LAP_test_mse)

0.057890626651083514 0.6077484857863533


In [21]:
LR_new_pred = LR.predict(X_new_normal)
LR_new = LR_new_pred*Y_train_std + Y_train_mean
print(LR_new)
print(np.around(LR_new))

[6.00469789 5.28767761 5.56363072 6.067022   5.94248207]
[6. 5. 6. 6. 6.]


In [22]:
RBF_new_pred = RBF.predict(X_new_normal)
RBF_new = RBF_new_pred*Y_train_std + Y_train_mean
print(RBF_new)
print(np.around(RBF_new))

[5.99233072 5.44373019 5.36230769 6.14112495 6.06319855]
[6. 5. 5. 6. 6.]


In [23]:
LAP_new_pred = LAP.predict(X_new_normal)
LAP_new = LAP_new_pred*Y_train_std + Y_train_mean
print(LAP_new)
print(np.around(LAP_new))

[6.0483042  5.47399545 5.62433419 5.97466709 6.00854608]
[6. 5. 6. 6. 6.]
